In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

def train_model():
    print(" Loading cleaned dataset ...")
    try:
        df = pd.read_csv("cleaned_earthquake_data.csv")
    except FileNotFoundError:
        print(" File 'cleaned_earthquake_data.csv' not found. Run data_cleaning.py first.")
        return

    print(f" Loaded {len(df)} records for training.")

    # Step 1 — Prepare features (X) and target (y)
    X = df[["Latitude", "Longitude", "Depth"]].values
    y = df["Magnitude"].values

    # Step 2 — Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Step 3 — Build neural network model
    model = Sequential([
        Dense(256, activation='tanh', input_shape=(3,)),
        Dropout(0.3),
        Dense(128, activation='tanh'),
        Dropout(0.2),
        Dense(64, activation='tanh'),
        Dense(1)
    ])

    # Step 4 — Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0008), loss='mean_absolute_error', metrics=['mae'])

    # Step 5 — Set up early stopping (to prevent overfitting)
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Step 6 — Train the model
    print(" Training the neural network ...")
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=64,
        callbacks=[early_stop],
        verbose=1
    )

    # Step 7 — Evaluate model performance
    print(" Evaluating model ...")
    y_pred = model.predict(X_test).flatten()

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print("\n Model Performance:")
    print(f"MAE  = {mae:.4f}")
    print(f"RMSE = {rmse:.4f}")
    print(f"R²   = {r2:.4f}")

    # Step 8 — Save model
    model.save("earthquake_model.h5")
    print(" Trained model saved as 'earthquake_model.h5'")

if __name__ == "__main__":
    train_model()

 Loading cleaned dataset ...
 Loaded 1000 records for training.
 Training the neural network ...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 3.9223 - mae: 3.9223 - val_loss: 1.5216 - val_mae: 1.5216
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1759 - mae: 1.1759 - val_loss: 1.0812 - val_mae: 1.0812
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0179 - mae: 1.0179 - val_loss: 0.6393 - val_mae: 0.6393
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6649 - mae: 0.6649 - val_loss: 0.5447 - val_mae: 0.5447
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5475 - mae: 0.5475 - val_loss: 0.4720 - val_mae: 0.4720
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4967 - mae: 0.4967 - val_loss: 0.4169 - val_mae: 0.4169
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4246 - mae: 0.4246 - val_loss: 0.3819 - val_mae: 0.3819
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3962 - mae: 0.3962 - val_loss: 0.3682 - val_mae: 0.3682
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3839 - mae: 0


 Model Performance:
MAE  = 0.3271
RMSE = 0.4788
R²   = -0.0986
 Trained model saved as 'earthquake_model.h5'
